In [ ]:
import arcpy, os
from arcpy.sa import *
from tqdm import tqdm
import pickle
import pandas as pd

In [ ]:
vars_hydro = [ "RC1", "RC2", "RC10", "RC1_O30", "RC1_O50", "RC1_C30", "RC1_C50", "RC1_P30", "RC1_P50", "RC2_O30", "RC2_O50", "RC2_C30", "RC2_C50", "RC2_P30", "RC2_P50"]
vars_chg = ["RC1_O3rc", "RC1_O5rc", "RC1_C3rc", "RC1_C5rc", "RC1_P3rc", "RC1_P5rc", "RC2_O3rc", "RC2_O5rc", "RC2_C3rc", "RC2_C5rc", "RC2_P3rc", "RC2_P5rc"]

In [ ]:
all_vars = vars_hydro + vars_chg
all_vars.append("area")

In [ ]:
# open pickled dictionaries
with open('grand_stats_1', 'rb') as infile:
    grand_dict1 = pickle.load(infile)
with open('grand_stats_2', 'rb') as infile:
    grand_dict2 = pickle.load(infile)
with open('grand_stats_3', 'rb') as infile:
    grand_dict3 = pickle.load(infile)
with open('grand_stats_fixed', 'rb') as infile:
    grand_dict3 = pickle.load(infile)

In [ ]:
# combine batched dictionaries
grand_dict = {**grand_dict1, **grand_dict2, **grand_dict3}

In [ ]:
# overwrite updated values -- these are catchments which needed revision after the first round

grand_dict.update(grand_dict_fixed)

In [ ]:
# function to create a dataframe from the dictionary

def make_table(dict_now, lyr, vars_now, type="grand"):
    table = pd.DataFrame.from_dict(dict_now, orient='index',  columns=all_vars)
    table = table.reset_index()
    table = table.rename(columns={'index':"dam_ID"})
    
    # get areas from dam point attribute tables
    areas = {}
    with arcpy.da.SearchCursor(lyr,vars_now) as cursor:
        for row in cursor:
            if type == "fhred":
                areas[str(row[0])[:-2]] = row[1]
            else:
                areas[str(row[0])] = row[1]
    table2 = pd.DataFrame.from_dict(areas, orient='index',  columns=['catch_skm'])
    table2 = table2.reset_index()
    table2 = table2.rename(columns={'index':"dam_ID"})
    
    # combine into one dataframe, compared calculated catchment area to given area
    table_results = pd.merge(table, table2, how="left", on="dam_ID")
    table_results['check_area'] = table_results['area'] - table_results['catch_skm']
    table_results['area_pct'] = table_results['check_area']/table_results['catch_skm']
    res = table['dam_ID'].tolist()
    res2 = table2['dam_ID'].tolist()
    for i in res2:
        if i not in res:
            print (i)
    return table_results

In [ ]:
# run function to create GRanD dataframe

grand_results = make_table(grand_dict, "GRanD_v1_3_selection", ['GRAND_ID', 'CATCH_SKM'])
print(len(grand_results))

In [ ]:
# check dataframe
grand_results.describe()

In [ ]:
# find all catchments which have a difference in catchment area of at least 2% and 20km2
# at the end there was only one of these remaining, which I checked manually and the catchment all falls within the same level 7 basin so it doesn't change the result

grand_problems = grand_results[((grand_results['area_pct'] > 0.02) | (grand_results['area_pct'] < -0.02)) & ((grand_results['check_area']> 20) | (grand_results['check_area'] <-20))]
grand_problems.sort_values(by='check_area')

In [ ]:
# find any results which could not be calculated (script to create dictionaries gave a value of -9999 in this case)
grand_results[grand_results.isin([-9999]).any(axis=1)] 

In [ ]:
# open fhred dictionaries and combine

with open('fhred_stats', 'rb') as infile:
    fhred0 = pickle.load(infile)
    print(len(fhred0))
with open('fhred_stats_1', 'rb') as infile:
    fhred1 = pickle.load(infile)
    print(len(fhred1))
with open('fhred_stats_2', 'rb') as infile:
    fhred2 = pickle.load(infile)
    print(len(fhred2))
with open('fhred_stats_3', 'rb') as infile:
    fhred3 = pickle.load(infile)
    print(len(fhred3))
fhred_dict = {**fhred0, **fhred1, **fhred2, **fhred3}
print(len(fhred_dict))

In [ ]:
# overwrite updated values
with open('fhred_stats_redo_all_final', 'rb') as infile:
    fhred_redo = pickle.load(infile)
    print(len(fhred_redo))
fhred_dict.update(fhred_redo)
print(len(fhred_dict))

In [ ]:
# create table of fhred results
fhred_results = make_table(fhred_dict, "FHReD2015_withGOID", ["DAM_ID", "UPLAND_SKM"], "fhred")

In [ ]:
# check dataframe
fhred_results.head()

In [ ]:
# check results make sense
fhred_results.describe()

In [ ]:
# find any results which could not be calculated (script to create dictionaries gave a value of -9999 in this case)
fhred_nd = fhred_results[fhred_results.isin([-9999]).any(axis=1)].sort_values(by='check_area')['dam_ID'].to_list()
print(fhred_nd)

In [ ]:
# check for errors by comparing calculated vs. given catchment areas
# I checked the remaining areas and they are small enough/within a level 7 basin to not make a difference in the risk calculations
fhred_problems = fhred_results[((fhred_results['area_pct'] > 0.05) | (fhred_results['area_pct'] < -0.05)) & ((fhred_results['check_area']> 20) | (fhred_results['check_area'] <-20))]
fhred_problems.sort_values(by='dam_ID')

In [ ]:
# create list of all dams which need to be redone
for i in fhred_problems['dam_ID'].to_list():
    if i not in fhred_nd:
        fhred_nd.append(i)